In [2]:
import numpy as np
import pandas as pd

In [3]:
movie = pd.read_csv("movies.csv",usecols=["movieId","title"],dtype={"movieId":"int32","title":"str"})
rating = pd.read_csv("ratings.csv",usecols=["userId","movieId","rating"] , dtype={"userId":"int32","movieId":"int32","rating":"float32"})

In [4]:
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
rating.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
df = pd.merge(movie,rating,on="movieId")
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [8]:
combine_movie_rating = df.dropna(axis=0,subset=["title"])
movie_rating_count = (combine_movie_rating.
                      groupby(by= ["title"])["rating"].
                      count().
                      reset_index().
                      rename(columns={"rating":"totalRatingCount"})[["title", "totalRatingCount"]])
movie_rating_count

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [9]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_rating_count , left_on="title" , right_on="title" , how="left")
rating_with_totalRatingCount.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [10]:
movie_rating_count["totalRatingCount"].describe()

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: totalRatingCount, dtype: float64

In [11]:
popularity_threshold = 50
rating_popular_movie = rating_with_totalRatingCount.query("totalRatingCount >= @popularity_threshold")
rating_popular_movie.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [12]:
rating_popular_movie.shape

(41362, 5)

In [14]:
movie_feature_df = rating_popular_movie.pivot_table(index="title",columns="userId",values="rating").fillna(0)
movie_feature_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [15]:
from scipy.sparse import csr_matrix
movie_feature_df_matrix = csr_matrix(movie_feature_df.values)
movie_feature_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
You've Got Mail (1998),0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
Young Frankenstein (1974),5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0


In [16]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(movie_feature_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [17]:
movie_feature_df_matrix.shape

(450, 606)

In [34]:
query_index = np.random.choice(movie_feature_df.shape[0])
print(query_index)
distances , indices = model_knn.kneighbors(movie_feature_df.iloc[query_index,:].values.reshape(1,-1) , n_neighbors=6)

196


In [35]:
for i in range(0, len(distances.flatten())):
    if i==0:
        print(f"Recommendations for {movie_feature_df.index[query_index]}:\n")
    else:
        print(f"{i}:{movie_feature_df.index[indices.flatten()[i]]}, with distance of {distances.flatten()[i]}")

Recommendations for Harry Potter and the Order of the Phoenix (2007):

1:Harry Potter and the Goblet of Fire (2005), with distance of 0.2362881898880005
2:Harry Potter and the Half-Blood Prince (2009), with distance of 0.28655409812927246
3:Harry Potter and the Chamber of Secrets (2002), with distance of 0.3290085196495056
4:Harry Potter and the Prisoner of Azkaban (2004), with distance of 0.336880624294281
5:Harry Potter and the Deathly Hallows: Part 2 (2011), with distance of 0.3396233320236206
